In [1]:
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable= False
    results = model.process(image)  #make prediction
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image.flags.writeable = True
    return image, results






In [5]:
def stylized_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color = (80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color = (80,110,10), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (110,212,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color = (110,212,10), thickness=2, circle_radius=2))
    
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (210,210,101), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color = (160,140,14), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (210,210,101), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color = (160,140,14), thickness=2, circle_radius=2))

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    

In [7]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        

        stylized_landmarks(image, results)
        imS = cv2.resize(image, (960, 540))    

        cv2.imshow("Live Feed", imS)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    


In [8]:
draw_landmarks(frame, results)

### EXTRACT KEYPOINT VALUES

In [9]:
len(results.pose_landmarks.landmark)

33

In [10]:
def extract_keypoints(results):
    pose = np.array([[i.x, i.y,i.z, i.visibility] for i in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lefthand = np.array([[i.x, i.y, i.z] for i in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    face = np.array([[i.x, i.y,i.z] for i in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    righthand = np.array([[i.x, i.y, i.z] for i in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lefthand, righthand])

In [11]:
len(results.pose_landmarks.landmark)  #33 elements, each element containing x,y,z, and visibility for pose...so we flatten it. 
                                      #In case, we dont have pose initially, so we multiply 4 by 33, giving 132
    
pose = np.array([[i.x, i.y,i.z, i.visibility] for i in results.pose_landmarks.landmark]).reshape(-1) if results.pose_landmarks else np.zeros(132)
len(pose)   #gives 32 as expected

132

In [12]:
result_test = extract_keypoints(results)
np.save("./0", result_test)
np.load('./0.npy', allow_pickle=True)

array([ 0.58693588,  0.90889299, -1.53378189, ...,  0.        ,
        0.        ,  0.        ])

### SETTING UP FOLDERS FOR DATA COLLECTION

In [13]:
path = os.path.join("MP_Data")

actions = np.array(['hello', 'thanks', 'iloveyou'])


no_sequences = 30          #30 videos worth of data for each action

sequence_length = 30       # each one of the videos are of 30 fps

### COLLECT KEYPOINTS FOR TRAINING AND TESTING

In [14]:
for i in actions:
    for j in range(no_sequences):
        try:
            os.makedirs(os.path.join(path, i, str(j)))
        except:
            pass
        
# 3 actions folder, each containing 30 videos per action and 30 frames per video

In [16]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    for action in actions:  #loop through each action
        for sequence in range(no_sequences):     #lopp through sequences meaning each video
            for frame_num in range(sequence_length):    #loop through sequence length i.e., frames per second

                ret, frame = cap.read()   #read feed

                image, results = mediapipe_detection(frame, holistic)   #make detections

                stylized_landmarks(image, results)   #draw landmarks with cute styles
                image = cv2.resize(image, (1300,600))   #resize
                
                if frame_num == 0:  #applying waiting logic
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, "Collecting frames for {} Video number/Sequence number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow("Live Feed", image)   #show image with stylized landmarks
                    cv2.waitKey(400)
                else:
                    cv2.putText(image, "Collecting frames for {} Video number/Sequence number {}".format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow("Live Feed", image)   #show image with stylized landmarks
                
                
                #exporting keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(path, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

### DATA PREPROCESSING AND FEATURE ENGINEERING

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:

label_map = {label:num for num, label in enumerate(actions)}

In [17]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [18]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(path, action, str(sequence), "{}.npy".format(frame_num)), allow_pickle=True)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [82]:
X = np.array(sequences)

In [83]:
X.shape

(90, 30, 1662)

In [84]:
y=to_categorical(labels)


In [85]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)


### BUILD AND CREATE LSTM NEURAL NETWORK

In [86]:
x_train.shape

(72, 30, 1662)

In [87]:
y_test.shape

(18, 3)

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [89]:
log_dir = os.path.join('./logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [90]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation = 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(512, activation='relu'))


model.add(Dense(actions.shape[0], activation='softmax'))




In [91]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('acc') is not None and logs.get('acc') > 0.80:
      print("\nReached 80% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [92]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [93]:
model.fit(x_train, y_train, epochs=500, callbacks=[callbacks])


Epoch 1/500
3/3 [==============================] - 7s 175ms/step - loss: 3.8049 - categorical_accuracy: 0.2917
Epoch 2/500
3/3 [==============================] - 1s 169ms/step - loss: 6.6049 - categorical_accuracy: 0.2917
Epoch 3/500
3/3 [==============================] - 1s 169ms/step - loss: 1.1616 - categorical_accuracy: 0.3611
Epoch 4/500
3/3 [==============================] - 1s 162ms/step - loss: 1.5154 - categorical_accuracy: 0.3611
Epoch 5/500
3/3 [==============================] - 1s 165ms/step - loss: 1.3625 - categorical_accuracy: 0.3611
Epoch 6/500
3/3 [==============================] - 1s 163ms/step - loss: 1.2446 - categorical_accuracy: 0.3611
Epoch 7/500
3/3 [==============================] - 1s 169ms/step - loss: 1.0528 - categorical_accuracy: 0.4444
Epoch 8/500
3/3 [==============================] - 1s 165ms/step - loss: 0.9526 - categorical_accuracy: 0.4861
Epoch 9/500
3/3 [==============================] - 1s 167ms/step - loss: 0.8718 - categorical_accuracy: 0.5833
E

3/3 [==============================] - 1s 175ms/step - loss: 1.0484 - categorical_accuracy: 0.6667
Epoch 75/500
3/3 [==============================] - 1s 206ms/step - loss: 1.4241 - categorical_accuracy: 0.5972
Epoch 76/500
3/3 [==============================] - 1s 179ms/step - loss: 0.8879 - categorical_accuracy: 0.6111
Epoch 77/500
3/3 [==============================] - 1s 185ms/step - loss: 0.6505 - categorical_accuracy: 0.7361
Epoch 78/500
3/3 [==============================] - 1s 179ms/step - loss: 0.6798 - categorical_accuracy: 0.7361
Epoch 79/500
3/3 [==============================] - 1s 192ms/step - loss: 0.5930 - categorical_accuracy: 0.7778
Epoch 80/500
3/3 [==============================] - 1s 191ms/step - loss: 0.6207 - categorical_accuracy: 0.7361
Epoch 81/500
3/3 [==============================] - 1s 173ms/step - loss: 0.5350 - categorical_accuracy: 0.8056
Epoch 82/500
3/3 [==============================] - 1s 178ms/step - loss: 0.5543 - categorical_accuracy: 0.7361
Epoch

3/3 [==============================] - 1s 163ms/step - loss: 0.5581 - categorical_accuracy: 0.7083
Epoch 147/500
3/3 [==============================] - 1s 158ms/step - loss: 0.4873 - categorical_accuracy: 0.8472
Epoch 148/500
3/3 [==============================] - 1s 167ms/step - loss: 0.4785 - categorical_accuracy: 0.8194
Epoch 149/500
3/3 [==============================] - 1s 166ms/step - loss: 0.4413 - categorical_accuracy: 0.8194
Epoch 150/500
3/3 [==============================] - 1s 167ms/step - loss: 0.3984 - categorical_accuracy: 0.8889
Epoch 151/500
3/3 [==============================] - 1s 167ms/step - loss: 0.3662 - categorical_accuracy: 0.8889
Epoch 152/500
3/3 [==============================] - 1s 165ms/step - loss: 0.3441 - categorical_accuracy: 0.8472
Epoch 153/500
3/3 [==============================] - 1s 164ms/step - loss: 0.3373 - categorical_accuracy: 0.8194
Epoch 154/500
3/3 [==============================] - 1s 169ms/step - loss: 0.3733 - categorical_accuracy: 0.86

3/3 [==============================] - 1s 176ms/step - loss: 0.0759 - categorical_accuracy: 0.9861
Epoch 219/500
3/3 [==============================] - 1s 158ms/step - loss: 0.1349 - categorical_accuracy: 0.9306
Epoch 220/500
3/3 [==============================] - 1s 162ms/step - loss: 0.1428 - categorical_accuracy: 0.9306
Epoch 221/500
3/3 [==============================] - 1s 160ms/step - loss: 0.1682 - categorical_accuracy: 0.9306
Epoch 222/500
3/3 [==============================] - 1s 170ms/step - loss: 0.1033 - categorical_accuracy: 0.9583
Epoch 223/500
3/3 [==============================] - 1s 156ms/step - loss: 0.2082 - categorical_accuracy: 0.9028
Epoch 224/500
3/3 [==============================] - 1s 166ms/step - loss: 0.1780 - categorical_accuracy: 0.9306
Epoch 225/500
3/3 [==============================] - 1s 163ms/step - loss: 0.2792 - categorical_accuracy: 0.8889
Epoch 226/500
3/3 [==============================] - 1s 166ms/step - loss: 0.2875 - categorical_accuracy: 0.86

3/3 [==============================] - 1s 173ms/step - loss: 0.0512 - categorical_accuracy: 0.9861
Epoch 291/500
3/3 [==============================] - 1s 164ms/step - loss: 0.0370 - categorical_accuracy: 1.0000
Epoch 292/500
3/3 [==============================] - 1s 167ms/step - loss: 0.0156 - categorical_accuracy: 1.0000
Epoch 293/500
3/3 [==============================] - 1s 160ms/step - loss: 0.0183 - categorical_accuracy: 1.0000
Epoch 294/500
3/3 [==============================] - 1s 169ms/step - loss: 0.0119 - categorical_accuracy: 1.0000
Epoch 295/500
3/3 [==============================] - 1s 161ms/step - loss: 0.0116 - categorical_accuracy: 1.0000
Epoch 296/500
3/3 [==============================] - 1s 168ms/step - loss: 0.0101 - categorical_accuracy: 1.0000
Epoch 297/500
3/3 [==============================] - 1s 178ms/step - loss: 0.0109 - categorical_accuracy: 1.0000
Epoch 298/500
3/3 [==============================] - 1s 171ms/step - loss: 0.0083 - categorical_accuracy: 1.00

3/3 [==============================] - 1s 162ms/step - loss: 4.8885e-04 - categorical_accuracy: 1.0000
Epoch 362/500
3/3 [==============================] - 1s 171ms/step - loss: 5.0084e-04 - categorical_accuracy: 1.0000
Epoch 363/500
3/3 [==============================] - 1s 160ms/step - loss: 5.9127e-04 - categorical_accuracy: 1.0000
Epoch 364/500
3/3 [==============================] - 1s 164ms/step - loss: 4.5062e-04 - categorical_accuracy: 1.0000
Epoch 365/500
3/3 [==============================] - 1s 164ms/step - loss: 4.8677e-04 - categorical_accuracy: 1.0000
Epoch 366/500
3/3 [==============================] - 1s 161ms/step - loss: 5.9445e-04 - categorical_accuracy: 1.0000
Epoch 367/500
3/3 [==============================] - 1s 163ms/step - loss: 4.5822e-04 - categorical_accuracy: 1.0000
Epoch 368/500
3/3 [==============================] - 1s 167ms/step - loss: 4.0028e-04 - categorical_accuracy: 1.0000
Epoch 369/500
3/3 [==============================] - 1s 167ms/step - loss: 4.4

3/3 [==============================] - 1s 172ms/step - loss: 1.2869e-04 - categorical_accuracy: 1.0000
Epoch 432/500
3/3 [==============================] - 1s 177ms/step - loss: 1.2679e-04 - categorical_accuracy: 1.0000
Epoch 433/500
3/3 [==============================] - 1s 167ms/step - loss: 1.2527e-04 - categorical_accuracy: 1.0000
Epoch 434/500
3/3 [==============================] - 1s 166ms/step - loss: 1.2229e-04 - categorical_accuracy: 1.0000
Epoch 435/500
3/3 [==============================] - 1s 161ms/step - loss: 1.2088e-04 - categorical_accuracy: 1.0000
Epoch 436/500
3/3 [==============================] - 1s 166ms/step - loss: 1.1928e-04 - categorical_accuracy: 1.0000
Epoch 437/500
3/3 [==============================] - 1s 166ms/step - loss: 1.1762e-04 - categorical_accuracy: 1.0000
Epoch 438/500
3/3 [==============================] - 1s 162ms/step - loss: 1.1633e-04 - categorical_accuracy: 1.0000
Epoch 439/500
3/3 [==============================] - 1s 162ms/step - loss: 1.1

In [94]:
model.load_weights('action.h5')

### PREDICTIONS

In [95]:
res = model.predict(x_test)

In [96]:
res

array([[2.1701628e-02, 9.6772456e-01, 1.0573786e-02],
       [8.5922760e-01, 1.7194765e-02, 1.2357768e-01],
       [2.9727989e-03, 9.9579573e-01, 1.2314854e-03],
       [5.2236724e-01, 1.9634930e-02, 4.5799783e-01],
       [6.1082006e-01, 2.8525624e-01, 1.0392372e-01],
       [4.3395522e-01, 7.6961413e-05, 5.6596786e-01],
       [9.8929606e-02, 8.9223158e-01, 8.8388482e-03],
       [8.9177221e-01, 5.3916883e-02, 5.4310892e-02],
       [9.2996389e-04, 9.5232934e-01, 4.6740625e-02],
       [1.2441862e-01, 8.3976364e-01, 3.5817817e-02],
       [3.2509959e-01, 4.6679258e-01, 2.0810783e-01],
       [1.1620686e-03, 5.6225836e-01, 4.3657961e-01],
       [2.2665139e-02, 9.7357506e-01, 3.7598556e-03],
       [1.1736296e-02, 8.6568147e-03, 9.7960693e-01],
       [6.1018032e-01, 4.7733117e-02, 3.4208652e-01],
       [6.4526314e-01, 5.8555730e-02, 2.9618105e-01],
       [8.2106555e-01, 3.4330457e-02, 1.4460407e-01],
       [6.2682074e-01, 2.6281649e-04, 3.7291643e-01]], dtype=float32)

In [97]:
actions[np.argmax(res[0])]

'thanks'

In [98]:
actions[np.argmax(y_test[0])]

'thanks'

### SAVING WEIGHTS
<br>

**We can save our model which will produce an m5 extension file. After tha, even if we delelte the model from our code, we can load it again using "load_weights" method. But before that, we need to do the following steps:**<br>

1- Build the model again.<br>
2- Compile the model.<br>
3- Then run the cell containing code for "load_weights".<br>

In [110]:
model.save('action.h5')

In [120]:
del model

In [100]:
model.load_weights("action.h5")

### MODEL EVALUATION

In [101]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(x_test)



***So see ,y_test and y_hat are noth one-hot encoded matrices containing outputs of the model. So what we do below is that we reconvert them from one hot encoding representations to their priginal format. For instance; at index 0 of y_test, we would have a vector [1,0,0] while in the index 0 of y_test after removing one-hot encoding would hold only the index of the number in y_test[0] which is equal to 1***

In [102]:
ytrue = np.argmax(y_test, axis=1)
yhat = np.argmax(yhat, axis=1)

In [103]:
multilabel_confusion_matrix(ytrue, yhat)


array([[[ 8,  2],
        [ 2,  6]],

       [[10,  2],
        [ 0,  6]],

       [[13,  1],
        [ 3,  1]]], dtype=int64)

In [109]:
print("Accuracy of model using accuracy metric is {}.".format(accuracy_score(ytrue, yhat)*100))

Accuracy of model using accuracy metric is 72.22222222222221.


In [108]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        stylized_landmarks(image, results)    #results will hold all landmarks on our image(face,hand etc)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)    #extract keypoints will basically extract the main keypoints
        sequence.append(keypoints)    
        sequence = sequence[-30:]   #taking the last 30 sequences, making 1 second 0f video
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]    #increasing the dimension of seqeucne to better suit for predction
            #print(actions[np.argmax(res)])       #res would hold three probabilities, argmax would extract the index of highest
                                                 #probability and then we would extract the element from action array corresponding to that index.
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

**We can see that the model has a test accuracy of 72.22 percent while training accuracy is 100%, so we can see that the model is overfitting and getting more data might be helpful.**